<a href="https://colab.research.google.com/github/jorgenotte/TiVA/blob/main/TiVA_FIGARO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importar librerías necesarias

import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Crear DataFrame con el mapeo de codigos nace y codigos geo
mapgeo=pd.read_excel('/content/drive/MyDrive/FIGARO/mapeo.xlsx','geo').set_index('cod')
mapnace=pd.read_excel('/content/drive/MyDrive/FIGARO/mapeo.xlsx','nace').set_index('cod')

# Convertir los DataFrame en diccionarios
mapgeo=dict(mapgeo.to_dict()['geo'])
mapnace=dict(mapnace.to_dict()['nace'])

In [4]:
# Leer el fichero en un dataframe [9363300 rows × 7 columns]
ruta='/content/drive/MyDrive/FIGARO/flatfile_eu-ic-io_ind-by-ind_23ed_2021.csv'
df=pd.read_csv(ruta)

In [5]:
df

,icioiRow,icioiCol,refArea,rowIi,counterpartArea,colIi,obsValue
0,AR_A01,AR_A01,AR,A01,AR,A01,9811.568
1,AR_A02,AR_A01,AR,A02,AR,A01,73.217
2,AR_A03,AR_A01,AR,A03,AR,A01,12.623
3,AR_B,AR_A01,AR,B,AR,A01,491.892
4,AR_C10T12,AR_A01,AR,C10T12,AR,A01,1488.984
...,...,...,...,...,...,...,...
9363295,W2_OP_RES,ZA_P5M,W2,OP_RES,ZA,P5M,0.000
9363296,W2_OP_NRES,ZA_P5M,W2,OP_NRES,ZA,P5M,0.000
9363297,W2_D1,ZA_P5M,W2,D1,ZA,P5M,0.000
9363298,W2_D29X39,ZA_P5M,W2,D29X39,ZA,P5M,0.000


In [6]:
# Se crea la columna agrega
df['agrega'] = df['refArea'].map(mapgeo) + '_' + df['rowIi'].map(mapnace) + '*' + df['counterpartArea'].map(mapgeo) + '_' + df['colIi'].map(mapnace)
df_colapsado=pd.DataFrame(df.groupby(['agrega'])['obsValue'].sum()).reset_index()

In [7]:
# Se separa la columna agrega en las columnas origen y destino
df_colapsado['origen']=df_colapsado['agrega'].str.split('*').str[0]
df_colapsado['destino']=df_colapsado['agrega'].str.split('*').str[1]

In [8]:
df_colapsado

,agrega,obsValue,origen,destino
0,ES_A*ES_A,2928.305,ES_A,ES_A
1,ES_A*ES_B,4.149,ES_A,ES_B
2,ES_A*ES_C,24086.185,ES_A,ES_C
3,ES_A*ES_D,47.292,ES_A,ES_D
4,ES_A*ES_E,15.316,ES_A,ES_E
...,...,...,...,...
5377,W2_OP_RES*RW_Q,0.000,W2_OP_RES,RW_Q
5378,W2_OP_RES*RW_R,0.000,W2_OP_RES,RW_R
5379,W2_OP_RES*RW_S,0.000,W2_OP_RES,RW_S
5380,W2_OP_RES*RW_T,0.000,W2_OP_RES,RW_T


In [9]:
# Se definen las variables del VA
valores1 = ['D21X31', 'OP_NRES', 'OP_RES', 'D1', 'D29X39', 'B2A3G']

# Se definen las variables de la demanda final
valores2 = ['P3_S13', 'P3_S14', 'P3_S15', 'P51G', 'P5M']

# Expresión regular que coincide con cualquiera de los caracteres exactos especificados
patron1 = '|'.join(valores1)
patron2 = '|'.join(valores2)

In [10]:
# Se crean los siguientes df: (z=Consumos intermedios) / (tls1=T-S s/productos) / (va=Value added)
df_z_pre = df_colapsado[~df_colapsado['origen'].str.contains(patron1, regex=True) & ~df_colapsado['destino'].str.contains(patron2, regex=True) ].reset_index(drop=True)
df_tls1 = df_colapsado[(df_colapsado['origen'] == 'W2_D21X31') & ~df_colapsado['destino'].str.contains(patron2, regex=True)].reset_index(drop=True)
df_va_pre = df_colapsado[((df_colapsado['origen'] == 'W2_D1') | (df_colapsado['origen'] == 'W2_D29X39') | (df_colapsado['origen'] == 'W2_B2A3G')) & ~df_colapsado['destino'].str.contains(patron2, regex=True)].reset_index(drop=True)

In [11]:
df_z_pre

,agrega,obsValue,origen,destino
0,ES_A*ES_A,2928.305,ES_A,ES_A
1,ES_A*ES_B,4.149,ES_A,ES_B
2,ES_A*ES_C,24086.185,ES_A,ES_C
3,ES_A*ES_D,47.292,ES_A,ES_D
4,ES_A*ES_E,15.316,ES_A,ES_E
...,...,...,...,...
3964,RW_U*RW_Q,0.000,RW_U,RW_Q
3965,RW_U*RW_R,0.000,RW_U,RW_R
3966,RW_U*RW_S,0.000,RW_U,RW_S
3967,RW_U*RW_T,0.000,RW_U,RW_T


In [12]:
# Se crea una nueva columna para conocer el total de CI
df_z_pre['agrega2'] = 'CI' + '*' + df_z_pre['destino']
df_z=pd.DataFrame(df_z_pre.groupby(['agrega2'])['obsValue'].sum()).reset_index()

# Se separa la columna agrega2 en las columnas origen y destino
df_z['origen']=df_z['agrega2'].str.split('*').str[0]
df_z['destino']=df_z['agrega2'].str.split('*').str[1]

In [13]:
# Se crea una nueva columna para conocer el total de VA
df_va_pre['agrega2'] = 'VA' + '*' + df_va_pre['destino']
df_va=pd.DataFrame(df_va_pre.groupby(['agrega2'])['obsValue'].sum()).reset_index()

# Se separa la columna agrega2 en las columnas origen y destino
df_va['origen']=df_va['agrega2'].str.split('*').str[0]
df_va['destino']=df_va['agrega2'].str.split('*').str[1]

In [14]:
# Se concatenan los df de CI, VA y T-S s/productos para conocer el total de X (Producción)
df_x_pre = pd.concat([df_z, df_va, df_tls1], ignore_index=True)

# Se crea una nueva columna para conocer el total de X (Producción)
df_x_pre['agrega2'] = 'X' + '*' + df_va_pre['destino']
df_x=pd.DataFrame(df_x_pre.groupby(['agrega2'])['obsValue'].sum()).reset_index()

# Se separa la columna agrega2 en las columnas origen y destino
df_x['origen']=df_x['agrega2'].str.split('*').str[0]
df_x['destino']=df_x['agrega2'].str.split('*').str[1]

In [15]:
# Se crean los siguientes df: (y=Demanda final) / (tls2=T-S s/productos) / (op=Purchases of non-residents in the domestic territory and direct purchase abroad by residents)
df_y = df_colapsado[~df_colapsado['origen'].str.contains(patron1, regex=True) & df_colapsado['destino'].str.contains(patron2, regex=True)].reset_index(drop=True)
df_tls2 = df_colapsado[(df_colapsado['origen'] == 'W2_D21X31') & df_colapsado['destino'].str.contains(patron2, regex=True)].reset_index(drop=True)
df_op = df_colapsado[((df_colapsado['origen'] == 'W2_OP_RES') | (df_colapsado['origen'] == 'W2_OP_NRES')) & df_colapsado['destino'].str.contains(patron2, regex=True)].reset_index(drop=True)

In [16]:
# Se crean los diferentes df en formato extendido
df_z_pre_tabla = df_z_pre.pivot_table(index="origen", columns="destino", values="obsValue")

df_z_tabla = df_z.pivot_table(index="origen", columns="destino", values="obsValue")

df_tls1_tabla = df_tls1.pivot_table(index="origen", columns="destino", values="obsValue")

df_va_tabla = df_va.pivot_table(index="origen", columns="destino", values="obsValue")

df_y_tabla = df_y.pivot_table(index="origen", columns="destino", values="obsValue")

df_tls2_tabla = df_tls2.pivot_table(index="origen", columns="destino", values="obsValue")

df_op_tabla = df_op.pivot_table(index="origen", columns="destino", values="obsValue")

df_x_tabla = df_x.pivot_table(index="origen", columns="destino", values="obsValue")

In [17]:
# Se extraen los valores numéricos de los diferentes df y se asignan a una matriz
matriz_z_pre = df_z_pre_tabla.values
matriz_z = df_z_tabla.values
matriz_tls1 = df_tls1_tabla.values
matriz_va = df_va_tabla.values
matriz_y = df_y_tabla.values
matriz_tls2 = df_tls2_tabla.values
matriz_op = df_op_tabla.values
matriz_x = df_x_tabla.values

In [18]:
# Value added to output ratio (VA/X)
np.seterr(divide='ignore', invalid='ignore')
matriz_ratio = np.where(matriz_x != 0, matriz_va / matriz_x, 0)

# Crear DataFrame y asignar nuevos índices
df_ratio = pd.DataFrame(matriz_ratio, columns=df_va_tabla.columns, index=df_va_tabla.index)
nuevo_indice = ['VA/X']
df_ratio.index = pd.Index(nuevo_indice, name='origen')

In [19]:
# Input coefficients (Z/X)
input_coeff = np.where(matriz_x != 0, matriz_z_pre / matriz_x, 0)

# Crear DataFrame y asignar nuevos índices
df_input_coeff = pd.DataFrame(input_coeff, columns=df_z_pre_tabla.columns, index=df_z_pre_tabla.index)

In [20]:
# Matriz identidad
num_filas = len(df_z_pre_tabla)
matriz_identidad = np.eye(num_filas)
df_matriz_identidad = pd.DataFrame(matriz_identidad, columns=df_z_pre_tabla.columns, index=df_z_pre_tabla.index)

# Diferencia entre matriz identidad y matriz de coeficientes técnicos
diferencia = matriz_identidad - input_coeff
df_diferencia = pd.DataFrame(diferencia, columns=df_z_pre_tabla.columns, index=df_z_pre_tabla.index)

# Matriz inversa de Leontief
inversa = np.linalg.inv(diferencia)
df_inversa = pd.DataFrame(inversa, columns=df_z_pre_tabla.columns, index=df_z_pre_tabla.index)

In [21]:
# Gross intermediate trade flows (GRTR_INT)

# Se divide las cadenas de texto que se encuentran en las columnas 'origen' y 'destino' en dos partes
df_z_pre[['origen2','x1']] = df_z_pre['origen'].str.split('_', n=1, expand=True)
df_z_pre[['destino2','x2']] = df_z_pre['destino'].str.split('_', n=1, expand=True)
df_z_pre = df_z_pre.drop(['x1','x2'], axis=1)

df_z_grtr = df_z_pre.copy()
df_z_grtr['obsValue2'] = np.where(df_z_grtr['origen2'] == df_z_grtr['destino2'], 0, df_z_grtr['obsValue'])
df_z_grtr['agrega2'] = df_z_grtr['origen'] + '*' + df_z_grtr['destino2']

df_z_grtr2=pd.DataFrame(df_z_grtr.groupby(['agrega2'])['obsValue2'].sum()).reset_index()
df_z_grtr2[['origen2','destino2']] = df_z_grtr2['agrega2'].str.split('*', n=1, expand=True)


In [22]:
df_z_grtr_table = df_z_grtr2.pivot_table(index="origen2", columns="destino2", values="obsValue2")
df_z_grtr_table

destino2,ES,FR,RW
origen2,,,
ES_A,0.000,1387.966,9530.994
ES_B,0.000,345.931,2382.272
ES_C,0.000,18769.316,102748.230
ES_D,0.000,333.000,827.591
ES_E,0.000,120.819,1986.080
...,...,...,...
RW_Q,128.480,267.075,0.000
RW_R,141.216,981.385,0.000
RW_S,85.901,309.498,0.000


In [23]:
# Gross final trade flows (GRTR_FNL)

# Se divide las cadenas de texto que se encuentran en las columnas 'origen' y 'destino' en dos partes
df_y[['origen2','x1']] = df_y['origen'].str.split('_', n=1, expand=True)
df_y[['destino2','x2']] = df_y['destino'].str.split('_', n=1, expand=True)
df_y = df_y.drop(['x1','x2'], axis=1)

df_y_grtr = df_y.copy()
df_y_grtr['obsValue2'] = np.where(df_y_grtr['origen2'] == df_y_grtr['destino2'], 0, df_y_grtr['obsValue'])
df_y_grtr['agrega2'] = df_y_grtr['origen'] + '*' + df_y_grtr['destino2']

df_y_grtr2=pd.DataFrame(df_y_grtr.groupby(['agrega2'])['obsValue2'].sum()).reset_index()
df_y_grtr2[['origen2','destino2']] = df_y_grtr2['agrega2'].str.split('*', n=1, expand=True)

In [24]:
df_y_grtr_table = df_y_grtr2.pivot_table(index="origen2", columns="destino2", values="obsValue2")
df_y_grtr_table

destino2,ES,FR,RW
origen2,,,
ES_A,0.000,2694.248,6603.589
ES_B,0.000,1.686,91.377
ES_C,0.000,15933.519,77750.166
ES_D,0.000,26.119,229.085
ES_E,0.000,20.737,127.586
...,...,...,...
RW_Q,44.936,345.154,0.000
RW_R,192.072,636.312,0.000
RW_S,68.104,145.076,0.000


In [25]:
# Gross total trade flows (GRTR_TOTAL)

df_zy_grtr_table = df_z_grtr_table + df_y_grtr_table
df_zy_grtr_table

destino2,ES,FR,RW
origen2,,,
ES_A,0.000,4082.214,16134.583
ES_B,0.000,347.617,2473.649
ES_C,0.000,34702.835,180498.396
ES_D,0.000,359.119,1056.676
ES_E,0.000,141.556,2113.666
...,...,...,...
RW_Q,173.416,612.229,0.000
RW_R,333.288,1617.697,0.000
RW_S,154.005,454.574,0.000


In [26]:
ruta_excel = '/content/drive/MyDrive/FIGARO/Indicadores TiVA.xlsx'

with pd.ExcelWriter(ruta_excel, engine='openpyxl') as writer:
    # Guardar cada DataFrame en una hoja diferente
    df_z_pre_tabla.to_excel(writer, sheet_name='Matriz CI', index=True)
    df_y_tabla.to_excel(writer, sheet_name='Matriz DF', index=True)
    df_tls1_tabla.to_excel(writer, sheet_name='Impuestos CI', index=True)
    df_tls2_tabla.to_excel(writer, sheet_name='Impuestos DF', index=True)
    df_va_tabla.to_excel(writer, sheet_name='VA', index=True)
    df_op_tabla.to_excel(writer, sheet_name='OP_RES_NRES', index=True)
    df_x_tabla.to_excel(writer, sheet_name='Producción', index=True)
    df_ratio.to_excel(writer, sheet_name='Ratio VA to X', index=True)
    df_z_grtr_table.to_excel(writer, sheet_name='GITF', index=True)
    df_y_grtr_table.to_excel(writer, sheet_name='GFTF', index=True)
    df_zy_grtr_table.to_excel(writer, sheet_name='GRTR_TOTAL', index=True)
    df_input_coeff.to_excel(writer, sheet_name='Input coefficient', index=True)
    df_matriz_identidad.to_excel(writer, sheet_name='Identity matrix', index=True)
    df_diferencia.to_excel(writer, sheet_name='I-A', index=True)
    df_inversa.to_excel(writer, sheet_name='Leontief inverse', index=True)

### **STRUCTURAL INDICATORS - BASED ON THE VALUES IN ICIO TABLES**

In [27]:
# 4.1 Production (Gross output) [OPCION-1]

production = round(df_x.loc[df_x['destino'].str.contains('ES'), 'obsValue'].sum(), 1)
print("La producción de ES es:", production)

production1 = round(df_x.loc[df_x['destino'].str.contains('FR'), 'obsValue'].sum(), 1)
print("La producción de FR es:", production1)

production2 = round(df_x.loc[df_x['destino'].str.contains('RW'), 'obsValue'].sum(), 1)
print("La producción de RW es:", production2)

La producción de ES es: 2134504.7
La producción de FR es: 4433156.5
La producción de RW es: 156233689.4


In [84]:
# 4.1 Production (Gross output) [OPCION-2]

indices_x_ES = [i for i, col in enumerate(df_x_tabla.columns) if 'ES' in col]
indices_x_FR = [i for i, col in enumerate(df_x_tabla.columns) if 'FR' in col]
indices_x_RW = [i for i, col in enumerate(df_x_tabla.columns) if 'RW' in col]

In [85]:
suma_prod_ES = np.round((np.sum(matriz_x[:, indices_x_ES], axis=1)), decimals=1)
suma_prod_FR = np.round((np.sum(matriz_x[:, indices_x_FR], axis=1)), decimals=1)
suma_prod_RW = np.round((np.sum(matriz_x[:, indices_x_RW], axis=1)), decimals=1)

In [86]:
print("La producción de ES es:", suma_prod_ES)
print("La producción de FR es:", suma_prod_FR)
print("La producción de RW es:", suma_prod_RW)

La producción de ES es: [2134504.7]
La producción de FR es: [4433156.5]
La producción de RW es: [1.56233689e+08]


In [87]:
# 4.1 Production (Gross output) [OPCION-3]

dic_matriz_x = {}
for fila in df_x_tabla.index:
    for columna in df_x_tabla.columns:
        dic_matriz_x[(fila, columna)] = matriz_x[df_x_tabla.index.get_loc(fila), df_x_tabla.columns.get_loc(columna)]

In [88]:
print(dic_matriz_x)

{('X', 'ES_A'): 60815.787000000004, ('X', 'ES_B'): 5652.944, ('X', 'ES_C'): 488610.57599999994, ('X', 'ES_D'): 65863.965, ('X', 'ES_E'): 29867.926, ('X', 'ES_F'): 157166.00100000002, ('X', 'ES_G'): 271195.97599999997, ('X', 'ES_H'): 117579.707, ('X', 'ES_I'): 87815.997, ('X', 'ES_J'): 91442.864, ('X', 'ES_K'): 81849.841, ('X', 'ES_L'): 149935.93800000002, ('X', 'ES_M'): 110281.809, ('X', 'ES_N'): 62977.765999999996, ('X', 'ES_O'): 98308.98800000001, ('X', 'ES_P'): 66773.956, ('X', 'ES_Q'): 118827.939, ('X', 'ES_R'): 32428.927999999996, ('X', 'ES_S'): 27580.834000000003, ('X', 'ES_T'): 9527.0, ('X', 'ES_U'): 0.0, ('X', 'FR_A'): 97018.484, ('X', 'FR_B'): 5022.838, ('X', 'FR_C'): 778766.112, ('X', 'FR_D'): 150671.185, ('X', 'FR_E'): 48033.953, ('X', 'FR_F'): 323530.83900000004, ('X', 'FR_G'): 464979.58400000003, ('X', 'FR_H'): 235049.807, ('X', 'FR_I'): 92977.387, ('X', 'FR_J'): 246953.74000000002, ('X', 'FR_K'): 249721.46899999995, ('X', 'FR_L'): 343346.36, ('X', 'FR_M'): 398646.696, ('X

In [89]:
# Se suman solo los valores donde la clave cumple cierta condición
suma_prod_ES = round(sum(valor for clave, valor in dic_matriz_x.items() if 'ES' in clave[1]),1)
print("La producción de ES es:", suma_prod_ES)

suma_prod_FR = round(sum(valor for clave, valor in dic_matriz_x.items() if 'FR' in clave[1]),1)
print("La producción de FR es:", suma_prod_FR)

suma_prod_RW = round(sum(valor for clave, valor in dic_matriz_x.items() if 'RW' in clave[1]),1)
print("La producción de RW es:", suma_prod_RW)

La producción de ES es: 2134504.7
La producción de FR es: 4433156.5
La producción de RW es: 156233689.4


In [31]:
# 4.2 Value added

value_added = round(df_va.loc[df_va['destino'].str.contains('ES'), 'obsValue'].sum(), 1)
print("El valor añadido de ES es:", value_added)

value_added1 = round(df_va.loc[df_va['destino'].str.contains('FR'), 'obsValue'].sum(), 1)
print("El valor añadido de FR es:", value_added1)

value_added2 = round(df_va.loc[df_va['destino'].str.contains('RW'), 'obsValue'].sum(), 1)
print("El valor añadido de RW es:", value_added2)

El valor añadido de ES es: 1090941.2
El valor añadido de FR es: 2216805.3
El valor añadido de RW es: 74148515.0


In [32]:
# 4.3 Value added as a % of production

df_ratio

destino,ES_A,ES_B,ES_C,ES_D,ES_E,ES_F,ES_G,ES_H,ES_I,ES_J,...,RW_L,RW_M,RW_N,RW_O,RW_P,RW_Q,RW_R,RW_S,RW_T,RW_U
origen,,,,,,,,,,,,,,,,,,,,,
VA/X,0.537515,0.410108,0.231397,0.283483,0.407522,0.427909,0.591551,0.421494,0.581569,0.478305,...,0.742609,0.535681,0.557126,0.632312,0.749561,0.605361,0.530847,0.588657,1.001546,0.0


In [33]:
df_ratio_pre = pd.melt(df_ratio, var_name='destino', value_name='obsValue')

df_filtrado1 = df_ratio_pre[df_ratio_pre['destino'].str.contains('ES')]

maximos = df_filtrado1.sort_values(by='obsValue', ascending=False).head(1)
minimos = df_filtrado1.sort_values(by='obsValue', ascending=True).head(1)
destino_max = maximos.iloc[0]['destino']
obsValue_max = maximos.iloc[0]['obsValue']
destino_min = minimos.iloc[0]['destino']
obsValue_min = minimos.iloc[0]['obsValue']
print("La rama de actividad de ES que presenta mayor ratio es:", destino_max)
print("Siendo su valor:", round(obsValue_max, 4))
print("La rama de actividad de ES que presenta menor ratio es:", destino_min)
print("Siendo su valor:", round(obsValue_min,4))


La rama de actividad de ES que presenta mayor ratio es: ES_T
Siendo su valor: 1.0
La rama de actividad de ES que presenta menor ratio es: ES_U
Siendo su valor: 0.0


In [34]:
# 4.4 Gross exports

# Intermediate products
int_exports1 = round(df_z_grtr.loc[(df_z_grtr['origen2'] == 'ES') & (df_z_grtr['destino2'] == 'FR'), 'obsValue2'].sum(), 1)
print("Las exportaciones de productos intermedios para ES con destino FR son:", int_exports1)

int_exports2 = round(df_z_grtr.loc[(df_z_grtr['origen2'] == 'ES') & (df_z_grtr['destino2'] == 'RW'), 'obsValue2'].sum(), 1)
print("Las exportaciones de productos intermedios para ES con destino RW son:", int_exports2)

# Final products
final_exports1 = round(df_y_grtr.loc[(df_y_grtr['origen2'] == 'ES') & (df_y_grtr['destino2'] == 'FR'), 'obsValue2'].sum(), 1)
print("Las exportaciones de productos terminados para ES con destino FR son:", final_exports1)

final_exports2 = round(df_y_grtr.loc[(df_y_grtr['origen2'] == 'ES') & (df_y_grtr['destino2'] == 'RW'), 'obsValue2'].sum(), 1)
print("Las exportaciones de productos terminados para ES con destino RW son:", final_exports2)

# Total gross exports
total_exports1 = round(int_exports1 + final_exports1, 1)
print("Las exportaciones totales para ES con destino FR son:", total_exports1)

total_exports2 = round(int_exports2 + final_exports2, 1)
print("Las exportaciones totales para ES con destino RW son:", total_exports2)

Las exportaciones de productos intermedios para ES con destino FR son: 33893.7
Las exportaciones de productos intermedios para ES con destino RW son: 202059.8
Las exportaciones de productos terminados para ES con destino FR son: 21958.3
Las exportaciones de productos terminados para ES con destino RW son: 121021.1
Las exportaciones totales para ES con destino FR son: 55852.0
Las exportaciones totales para ES con destino RW son: 323080.9


In [35]:
# 4.5 Gross imports

# Intermediate products
int_imports1 = round(df_z_grtr.loc[(df_z_grtr['origen2'] == 'FR') & (df_z_grtr['destino2'] == 'ES'), 'obsValue2'].sum(), 1)
print("Las importaciones de productos intermedios para ES procedentes de FR son:", int_imports1)

int_imports2 = round(df_z_grtr.loc[(df_z_grtr['origen2'] == 'RW') & (df_z_grtr['destino2'] == 'ES'), 'obsValue2'].sum(), 1)
print("Las importaciones de productos intermedios para ES procedentes de RW son:", int_imports2)

# Final products
final_imports1 = round(df_y_grtr.loc[(df_y_grtr['origen2'] == 'FR') & (df_y_grtr['destino2'] == 'ES'), 'obsValue2'].sum(), 1)
print("Las importaciones de productos terminados para ES procedentes de FR son:", final_imports1)

final_imports2 = round(df_y_grtr.loc[(df_y_grtr['origen2'] == 'RW') & (df_y_grtr['destino2'] == 'ES'), 'obsValue2'].sum(), 1)
print("Las importaciones de productos terminados para ES procedentes de RW son:", final_imports2)

# Total gross imports
total_imports1 = round(int_imports1 + final_imports1, 1)
print("Las importaciones totales para ES procedentes de FR son:", total_imports1)
total_imports2 = round(int_imports2 + final_imports2, 1)
print("Las importaciones totales para ES procedentes de RW son:", total_imports2)

Las importaciones de productos intermedios para ES procedentes de FR son: 27561.3
Las importaciones de productos intermedios para ES procedentes de RW son: 220044.5
Las importaciones de productos terminados para ES procedentes de FR son: 12939.2
Las importaciones de productos terminados para ES procedentes de RW son: 117173.1
Las importaciones totales para ES procedentes de FR son: 40500.5
Las importaciones totales para ES procedentes de RW son: 337217.6


In [36]:
# 4.6 Gross trade balance

# Difference between Gross Exports and Gross Imports
gross_trade_balance1 = round(total_exports1 - total_imports1, 1)
print("El balance de exportaciones/importaciones de ES con FR son:", gross_trade_balance1)
gross_trade_balance2 = round(total_exports2 - total_imports2, 1)
print("El balance de exportaciones/importaciones de ES con RW son:", gross_trade_balance2)

El balance de exportaciones/importaciones de ES con FR son: 15351.5
El balance de exportaciones/importaciones de ES con RW son: -14136.7


In [37]:
# 4.7 Gross exports, partner shares (%) (ES-FR)

df_z_grtr1 = df_z_grtr.loc[(df_z_grtr['origen'].str.contains('ES')) & (df_z_grtr['destino2'] == 'FR')].reset_index(drop=True)
df_z_grtr1=pd.DataFrame(df_z_grtr1.groupby(['agrega2'])['obsValue2'].sum()).reset_index()
df_z_grtr1['gross exports %'] = (df_z_grtr1['obsValue2'] / int_exports1)*100
df_z_grtr1

,agrega2,obsValue2,gross exports %
0,ES_A*FR,1387.966,4.095056
1,ES_B*FR,345.931,1.020635
2,ES_C*FR,18769.316,55.377005
3,ES_D*FR,333.000,0.982483
4,ES_E*FR,120.819,0.356464
5,ES_F*FR,152.697,0.450517
6,ES_G*FR,6537.066,19.286965
7,ES_H*FR,2091.011,6.169321
8,ES_I*FR,689.628,2.034679
9,ES_J*FR,522.999,1.543057


In [38]:
# 4.7 Gross exports, partner shares (%) (ES-RW)

df_z_grtr2 = df_z_grtr.loc[(df_z_grtr['origen'].str.contains('ES')) & (df_z_grtr['destino2'] == 'RW')].reset_index(drop=True)
df_z_grtr2=pd.DataFrame(df_z_grtr2.groupby(['agrega2'])['obsValue2'].sum()).reset_index()
df_z_grtr2['gross exports %'] = (df_z_grtr2['obsValue2'] / int_exports2)*100
df_z_grtr2

,agrega2,obsValue2,gross exports %
0,ES_A*RW,9530.994,4.716917
1,ES_B*RW,2382.272,1.178994
2,ES_C*RW,102748.230,50.850407
3,ES_D*RW,827.591,0.409577
4,ES_E*RW,1986.080,0.982917
5,ES_F*RW,1459.858,0.722488
6,ES_G*RW,38277.902,18.943848
7,ES_H*RW,12183.567,6.029684
8,ES_I*RW,2273.080,1.124954
9,ES_J*RW,7080.433,3.504127


In [39]:
# 4.8 Gross imports, partner shares (%) (FR-ES)

df_z_grtr3 = df_z_grtr.loc[(df_z_grtr['origen'].str.contains('FR')) & (df_z_grtr['destino2'] == 'ES')].reset_index(drop=True)
df_z_grtr3=pd.DataFrame(df_z_grtr3.groupby(['agrega2'])['obsValue2'].sum()).reset_index()
df_z_grtr3['gross exports %'] = (df_z_grtr3['obsValue2'] / int_imports1)*100
df_z_grtr3

,agrega2,obsValue2,gross exports %
0,FR_A*ES,739.821,2.684275
1,FR_B*ES,156.424,0.567549
2,FR_C*ES,14442.615,52.401792
3,FR_D*ES,735.981,2.670342
4,FR_E*ES,840.721,3.050368
5,FR_F*ES,0.045,0.000163
6,FR_G*ES,3720.080,13.497477
7,FR_H*ES,2552.509,9.261207
8,FR_I*ES,0.000,0.000000
9,FR_J*ES,858.825,3.116054


In [40]:
# 4.8 Gross imports, partner shares (%) (RW-ES)

df_z_grtr4 = df_z_grtr.loc[(df_z_grtr['origen'].str.contains('RW')) & (df_z_grtr['destino2'] == 'ES')].reset_index(drop=True)
df_z_grtr4=pd.DataFrame(df_z_grtr4.groupby(['agrega2'])['obsValue2'].sum()).reset_index()
df_z_grtr4['gross exports %'] = (df_z_grtr4['obsValue2'] / int_imports2)*100
df_z_grtr4

,agrega2,obsValue2,gross exports %
0,RW_A*ES,4144.088,1.883295
1,RW_B*ES,20106.711,9.137566
2,RW_C*ES,109707.330,49.856883
3,RW_D*ES,1542.648,0.701062
4,RW_E*ES,2309.065,1.049363
5,RW_F*ES,682.366,0.310104
6,RW_G*ES,28176.916,12.805099
7,RW_H*ES,13538.402,6.152575
8,RW_I*ES,2935.341,1.333976
9,RW_J*ES,7996.556,3.634063


In [41]:
# 5.1 Domestic value added content of gross exports

# Filtrar el DataFrame por filas y columnas que contienen "ES"
df_inversa_ES = df_inversa.loc[df_inversa.index.str.contains('ES'), df_inversa.columns.str.contains('ES')]

# Filtrar el DataFrame por filas y columnas que contienen "ES"
df_ratio_ES = df_ratio.loc[df_ratio.index, df_ratio.columns.str.contains('ES')]

matriz_inversa_ES = df_inversa_ES.values
matriz_ratio_ES = df_ratio_ES.values

a = np.dot(matriz_ratio_ES, matriz_inversa_ES)
a_transpuesta = np.transpose(a)

In [42]:
df_zy_grtr_table_filtrado1 = df_zy_grtr_table.loc[df_zy_grtr_table.index.str.contains('ES'), df_zy_grtr_table.columns.str.contains('FR')]
matriz_export_ES_FR = df_zy_grtr_table_filtrado1.values

df_zy_grtr_table_filtrado2 = df_zy_grtr_table.loc[df_zy_grtr_table.index.str.contains('ES'), df_zy_grtr_table.columns.str.contains('RW')]
matriz_export_ES_RW = df_zy_grtr_table_filtrado2.values

In [43]:
exgr_dva_ES_FR = a_transpuesta * matriz_export_ES_FR
exgr_dva_ES_WR = a_transpuesta * matriz_export_ES_RW

In [44]:
exgr_dva_ES_FR

array([[ 3404.89531639],
       [  262.63184117],
       [19762.70905176],
       [  229.73792367],
       [  106.97082178],
       [  148.40617628],
       [ 6643.35952676],
       [ 2140.36713932],
       [  652.41276979],
       [ 1157.53203387],
       [  472.86843061],
       [   30.39566544],
       [ 1463.24273852],
       [  967.37362568],
       [  231.05300839],
       [   47.44608528],
       [   26.61438608],
       [   76.67022667],
       [   70.86886287],
       [    0.        ],
       [    0.        ]])

In [45]:
exgr_dva_ES_WR

array([[ 13457.54193401],
       [  1868.89303825],
       [102790.94732341],
       [   675.98358825],
       [  1597.25189316],
       [  1831.98187195],
       [ 44376.33290602],
       [ 13589.26492651],
       [  4798.84053149],
       [ 11634.52765544],
       [  5706.92341231],
       [   272.22741311],
       [ 13190.87959867],
       [  5842.84925065],
       [  1997.0224076 ],
       [   364.14069645],
       [   295.80524657],
       [   682.34743822],
       [   475.75865251],
       [     0.        ],
       [     0.        ]])

In [46]:
# 5.2 Domestic value added share of gross exports

suma_numerador = exgr_dva_ES_FR + exgr_dva_ES_WR
suma_denominador = matriz_export_ES_FR + matriz_export_ES_RW

exgr_dvash = (suma_numerador / suma_denominador) *100

In [47]:
# 5.3 Industry domestic value added contribution to gross exports

suma_numerador = exgr_dva_ES_FR + exgr_dva_ES_WR
suma_denominador = np.sum(matriz_export_ES_FR) + np.sum(matriz_export_ES_RW)

exgr_dtvaind = (suma_numerador / suma_denominador) *100

In [48]:
# 5.4 Domestic value added in gross exports, partner shares

numerador1 = exgr_dva_ES_FR
numerador2 = exgr_dva_ES_WR
suma_denominador = exgr_dva_ES_FR + exgr_dva_ES_WR

exgr_dvapsh1 = (numerador1 / suma_denominador) *100
exgr_dvapsh2 = (numerador2 / suma_denominador) *100

In [49]:
# 5.5 Direct domestic industry value added content of gross exports

diagonal = np.diag(matriz_inversa_ES)
nueva_matriz = np.diag(diagonal)
a =np.dot(matriz_ratio_ES, nueva_matriz)
a_transpuesta = np.transpose(a)

In [50]:
exgr_ddc = a_transpuesta * (matriz_export_ES_FR + matriz_export_ES_RW)
exgr_ddc

array([[11548.58956414],
       [ 1161.0319461 ],
       [64427.65798465],
       [  580.63098911],
       [ 1119.88696223],
       [ 1289.74806321],
       [35368.24338817],
       [10965.04806895],
       [ 3710.9220148 ],
       [ 8831.07257315],
       [ 4968.83709685],
       [  272.63556372],
       [11369.01691652],
       [ 5573.55660194],
       [ 1819.78458945],
       [  378.42453474],
       [  268.61246634],
       [  580.08933789],
       [  437.19677861],
       [    0.        ],
       [    0.        ]])

In [51]:
# 5.6 Indirect domestic value added content of gross exports (originating from domestic intermediates)

nueva_matriz = matriz_inversa_ES.copy()
np.fill_diagonal(nueva_matriz, 0)
a =np.dot(matriz_ratio_ES, nueva_matriz)
a_transpuesta = np.transpose(a)

In [52]:
exgr_idc = a_transpuesta * (matriz_export_ES_FR + matriz_export_ES_RW)
exgr_idc

array([[5.31384769e+03],
       [9.70492933e+02],
       [5.81259984e+04],
       [3.25090523e+02],
       [5.84335753e+02],
       [6.90639985e+02],
       [1.56514490e+04],
       [4.76458400e+03],
       [1.74033129e+03],
       [3.96098712e+03],
       [1.21095475e+03],
       [2.99875148e+01],
       [3.28510542e+03],
       [1.23666627e+03],
       [4.08290827e+02],
       [3.31622470e+01],
       [5.38071663e+01],
       [1.78928327e+02],
       [1.09430737e+02],
       [0.00000000e+00],
       [0.00000000e+00]])

In [53]:
exgr_dva_ES_FR + exgr_dva_ES_WR

array([[ 16862.4372504 ],
       [  2131.52487942],
       [122553.65637517],
       [   905.72151192],
       [  1704.22271494],
       [  1980.38804823],
       [ 51019.69243278],
       [ 15729.63206583],
       [  5451.25330128],
       [ 12792.0596893 ],
       [  6179.79184292],
       [   302.62307855],
       [ 14654.12233719],
       [  6810.22287634],
       [  2228.07541599],
       [   411.58678173],
       [   322.41963265],
       [   759.0176649 ],
       [   546.62751538],
       [     0.        ],
       [     0.        ]])

In [54]:
exgr_ddc

array([[11548.58956414],
       [ 1161.0319461 ],
       [64427.65798465],
       [  580.63098911],
       [ 1119.88696223],
       [ 1289.74806321],
       [35368.24338817],
       [10965.04806895],
       [ 3710.9220148 ],
       [ 8831.07257315],
       [ 4968.83709685],
       [  272.63556372],
       [11369.01691652],
       [ 5573.55660194],
       [ 1819.78458945],
       [  378.42453474],
       [  268.61246634],
       [  580.08933789],
       [  437.19677861],
       [    0.        ],
       [    0.        ]])

In [55]:
exgr_idc

array([[5.31384769e+03],
       [9.70492933e+02],
       [5.81259984e+04],
       [3.25090523e+02],
       [5.84335753e+02],
       [6.90639985e+02],
       [1.56514490e+04],
       [4.76458400e+03],
       [1.74033129e+03],
       [3.96098712e+03],
       [1.21095475e+03],
       [2.99875148e+01],
       [3.28510542e+03],
       [1.23666627e+03],
       [4.08290827e+02],
       [3.31622470e+01],
       [5.38071663e+01],
       [1.78928327e+02],
       [1.09430737e+02],
       [0.00000000e+00],
       [0.00000000e+00]])

In [56]:
# 5.7 Reimported domestic value added content of gross exports

exgr_rim = (exgr_dva_ES_FR + exgr_dva_ES_WR) - exgr_ddc - exgr_idc
exgr_rim

array([[-9.09494702e-13],
       [ 2.27373675e-13],
       [ 7.27595761e-12],
       [-1.70530257e-13],
       [ 2.27373675e-13],
       [ 0.00000000e+00],
       [-5.45696821e-12],
       [ 0.00000000e+00],
       [-6.82121026e-13],
       [-2.27373675e-12],
       [ 0.00000000e+00],
       [-1.42108547e-14],
       [ 1.81898940e-12],
       [ 0.00000000e+00],
       [-5.68434189e-14],
       [-1.20792265e-13],
       [ 4.97379915e-14],
       [-5.68434189e-14],
       [-4.26325641e-14],
       [ 0.00000000e+00],
       [ 0.00000000e+00]])